In [2]:
import duckdb

In [10]:
# Paths for the fact and dimensions tables
dim_dates = r'../data/analytics/dim_dates.csv'
dim_users = r'../data/analytics/dim_users.csv'
fact_events = r'../data/analytics/fact_events.csv'
fact_transactions = r'../data/analytics/fact_transactions.csv'

In [4]:
# Create a DuckDB connection
con = duckdb.connect(database=':memory:')

In [12]:
# Load the CSV files into DuckDB
con.execute(f"""
CREATE OR REPLACE TABLE dim_dates AS SELECT * FROM read_csv_auto('{dim_dates}');
CREATE OR REPLACE TABLE dim_users AS SELECT * FROM read_csv_auto('{dim_users}');
CREATE OR REPLACE TABLE fact_events AS SELECT * FROM read_csv_auto('{fact_events}');
CREATE OR REPLACE TABLE fact_transactions AS SELECT * FROM read_csv_auto('{fact_transactions}');
""")

1. How many users were active on a given day (they made a deposit or withdrawal)

In [18]:
query = """
SELECT 
    event_timestamp::date as date,
    COUNT(DISTINCT user_id) as active_users
FROM fact_transactions
GROUP BY date;
"""
con.execute(query).fetchdf()

,date,active_users
0,NaT,70
1,2022-10-05,18
2,2022-01-06,10
3,2022-01-12,2
4,2022-01-31,3
...,...,...
1017,2023-04-18,1
1018,2020-06-13,1
1019,2022-11-29,1
1020,2023-03-20,1


2. Identify users haven't made a deposit

In [19]:
query = """
SELECT 
    user_id
FROM 
    dim_users
WHERE 
    user_id NOT IN (SELECT DISTINCT user_id FROM fact_transactions WHERE transaction_type = 'deposit');
"""
con.execute(query).fetchdf()

,user_id
0,f3971ac4ef3eb040dadaeed846ac6b4e
1,2eef370c267c2f741b744b27741d5c90
2,4b28b220ac44eb9727640e0b78cc622c
3,c08f0c801bd312c3d2358c1ed0bc1bea
4,8b48e30332fc417534491ce3fda913b9
...,...
12226,75743dbfcf177f2885a59b460017feee
12227,f24b649a42a0f7825ddbbfeff6a9a675
12228,b55d7ce2adb9449fc4dae6115cbbe30f
12229,bca050a896b3070dba891b0a4404aacd


3. Identify on a given day which users have made more than 5 deposits historically

In [21]:
query = """
SELECT 
    user_id
FROM 
    fact_transactions
WHERE 
    transaction_type = 'deposit'
    AND event_timestamp::date = '2020-01-16'
GROUP BY user_id
HAVING COUNT(transaction_id) > 5;
"""
con.execute(query).fetchdf()

,user_id
0,d6288499d0083cc34e60a077b7c4b3e1


4. When was the last time a user made a login

In [25]:
query = """
SELECT
    user_id,
    MAX(event_timestamp) as last_login
FROM 
    fact_events
WHERE
    LOWER(event_name) LIKE '%login%'
GROUP BY 1;
"""
con.execute(query).fetchdf()

,user_id,last_login
0,020bf2c45e7bb322f89a226bd2c5d41b,2020-02-19 18:45:26.390
1,1fb2a1c37b18aa4611c3949d6148d0f8,2020-02-24 22:02:07.299
2,cf05968255451bdefe3c5bc64d550517,2020-02-25 23:56:04.859
3,dc58e3a306451c9d670adcd37004f48f,2023-01-30 22:28:57.399
4,1145a30ff80745b56fb0cecf65305017,2022-11-09 01:26:17.825
...,...,...
24184,ba0909e302db12f30293fed31693f19b,2023-08-22 07:44:26.218
24185,cb2dc70c84b332c8e5aef3045e20c16f,2023-08-22 08:50:18.126
24186,53fcf303d2e920fd3c90921ceef16a92,2023-08-22 19:10:15.627
24187,c1dc18bcfca3ef19cb61c20dd5c6e931,2023-08-22 21:30:41.655


5. How many times a user has made a login between two dates

In [32]:
query = """
SELECT
    user_id,
    COUNT(*) as login_count
FROM 
    fact_events
WHERE
    LOWER(event_name) LIKE '%login%'
    AND event_timestamp::date BETWEEN '2020-06-01' AND '2022-12-31'
GROUP BY user_id;
"""
con.execute(query).fetchdf()

,user_id,login_count
0,3e33b970f21d2fc65096871ea0d2c6e4,1
1,e91068fff3d7fa1594dfdf3b4308433a,29
2,ec0f40c389aeef789ce03eb814facc6c,1
3,728f206c2a01bf572b5940d7d9a8fa4c,185
4,93da579a65ce84cd1d4c85c2cbb84fc5,1
...,...,...
17700,d5882d22ec8a123cd6e4a50432049d67,1
17701,6d59dac9480611cf3e9e9b0d64ec2cff,1
17702,7363e6c48d2c5eab196b0080045d0cbd,1
17703,5adb9efda92501648f9dcc408a5e9b11,1


6. Number of unique currencies deposited on a given day

In [34]:
query = """
SELECT 
    event_timestamp::date as date, 
    COUNT(DISTINCT currency) as unique_currencies
FROM 
    fact_transactions
WHERE 
    transaction_type = 'deposit'
GROUP BY date;
"""
con.execute(query).fetchdf()

,date,unique_currencies
0,2022-02-03,4
1,2022-05-04,5
2,2022-06-26,2
3,2022-07-13,52
4,2022-11-03,46
...,...,...
878,2021-09-18,1
879,2020-05-17,1
880,2021-04-10,1
881,2020-04-02,1


7. Number of unique currencies withdrew on a given day

In [36]:
query = """
SELECT 
    event_timestamp::date as date, 
    COUNT(DISTINCT currency) as unique_currencies
FROM 
    fact_transactions
WHERE 
    transaction_type = 'withdrawal'
GROUP BY date;
"""
con.execute(query).fetchdf()

,date,unique_currencies
0,2020-03-11,3
1,2020-03-21,1
2,2020-04-06,2
3,2020-03-19,4
4,2020-05-29,1
...,...,...
783,2022-05-03,1
784,2021-08-10,1
785,2021-08-12,1
786,2022-05-22,1


8. Total amount deposited of a given currency on a given day

In [37]:
query = """
SELECT 
    event_timestamp::date as date, 
    currency, 
    SUM(amount) as total_amount
FROM 
    fact_transactions
WHERE 
    transaction_type = 'deposit'
GROUP BY date, currency;
"""
con.execute(query).fetchdf()

,date,currency,total_amount
0,2020-01-06,ltc,20.000000
1,2020-01-06,mxn,650000.000000
2,2020-01-27,bsv,200.000000
3,2020-01-27,btc,200.000000
4,2020-03-13,xrp,12.000000
...,...,...,...
7902,2022-01-17,ltc,0.083689
7903,2022-01-17,mxn,5780.229611
7904,2022-01-19,bsv,1000.000000
7905,2022-01-19,btc,1000.000000
